In [1]:
from utils import *
import numpy as np
import time
import pprint
import plotly.io as pio
import plotly.offline as pyo
from matplotlib import pyplot as plt
import geopandas as gpd
from shapely.geometry import Polygon
from shapely.ops import unary_union
pyo.init_notebook_mode(connected=True)
np.set_printoptions(suppress=True)
#pio.renderers.default = 'browser'
pp = pprint.PrettyPrinter(indent=4)

ModuleNotFoundError: No module named 'cartopy'

In [ ]:
# Set the latitude and longitude of the center point
lat = 51.5074
lon = 0.1278
radius = 300

In [ ]:
# Find all the buildings in the given search location
data = find_buildings(lat, lon, radius)

In [ ]:
# Populate the dict 'buildings' with all the relevant data
buildings = populate_buildings(data)
#pp.pprint(buildings)

In [ ]:
# Convert the node numbers to latitude and longitude coordinates (slow!)
buildings = convert_node_to_coords(buildings, plot=False)

In [ ]:
# Create a list of building instances that we can work with
bN = [building(building_from_ind(buildings, i)[1]) for i in range(len(buildings))]

t1 = building_from_ind(buildings, 0)[0]

In [ ]:
# Calculate the sun's vector 
sunpath = calc_sunvector(t1, (2022, 6, 21, 12, 0, 0, 0))

bN = [building(building_from_ind(buildings, i)[1]) for i in range(len(buildings))]

start = time.time()
calculate_shadows(bN, sunpath)
end = time.time()

print(f"Finished calculating shadows! Time taken: {end-start}s")

shad = [pd.DataFrame(b.overall_shadow_poly,columns=["lat", "lon", "height"])/111139  for b in bN]

plotly_buildings(buildings, shad, zoom=15)

In [ ]:
%matplotlib notebook
    
plot_rays_vertices(bN, s=50, figsize=(10,10), extent_offset=50)

In [ ]:
# Visualise the buildings that have been captured
plotly_buildings(buildings)

In [ ]:
def combine_shadow_polys(b):
    # Create Polygons for each face that's projected onto the floor
    polys = [Polygon([tuple(point) for point in s]) for s in b.shadow_polys]

    # Merge the Polygons together
    mergedPolys = unary_union(polys)

    # Extract the outline of the new Polygon
    xx, yy = mergedPolys.exterior.coords.xy
    xx = xx.tolist()
    yy = yy.tolist()

    #gpd.GeoSeries([mergedPolys]).boundary.plot()
    #plt.show()
    #mergedPolys = unary_union(polys)
    
    return np.array([xx, yy]).reshape(2,-1).T
    
    

In [ ]:
def plot_rays_vertices(building,
              s=50,
              extent_offset=20,
              figsize=(15,15)):

    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(111, projection='3d')
    
    def plot_polys(building):
        shadow_poly = Poly3DCollection([[tuple(point) for point in building.overall_shadow_poly]], alpha=0.2, color="black")
        ax.add_collection3d(shadow_poly)

        for i, face in enumerate(building.faces):
            vertices = [[tuple(v) for v in face]]
            poly = Poly3DCollection(vertices, alpha=0.8)
            ax.add_collection3d(poly)

    if isinstance(building, list):
        for b in building:
            plot_polys(b)
        all_faces = np.concatenate(np.array([np.concatenate(np.array(b.faces), dtype=object) for b in building]))
    else:
        plot_polys(building)
        all_faces = np.concatenate(np.array(b.faces), dtype=object)
    
    mins = all_faces.min(axis=0)
    maxs = all_faces.max(axis=0)
    
    ax.set_xlim(mins[0]-extent_offset,maxs[0]+extent_offset)
    ax.set_ylim(mins[1]-extent_offset,maxs[1]+extent_offset)
    ax.set_zlim(mins[2]-extent_offset,maxs[2]+extent_offset) 

In [ ]:
def calculate_shadows(bN):
    for b in bN:
        b.collision_coords = []
        b.shadow_coords = []
        b.origin_coords = []
        b.sunny_coords = []  
        b.shadow_con_hull = []
        b.shadow_polys = []

        for face in b.faces:
            shadow_coords = []
            for i, O in enumerate(face):
                ground_level = 0
                L = (ground_level-O[2])/sunpath[2]  # How many sunpaths do we need to move before we hit the ground?
                D = O + L*sunpath
                b.collision_coords.append(O)
                shadow_coords.append(D)
                b.origin_coords.append(O)
                b.sunny_coords.append([])

            shadow = np.array(shadow_coords)
            b.shadow_polys.append(shadow)

            b.overall_shadow_poly = combine_shadow_polys(b)

            removed_coord = np.full(len(b.overall_shadow_poly),0)
            b.overall_shadow_poly = np.column_stack((b.overall_shadow_poly, removed_coord))

In [ ]:
# Populate the building_collection class with all the buildings that have been found
bN = [building(building_from_ind(buildings, i)[1]) for i in range(len(buildings))]
bN = building_collection(bN)

# Populate a DataFrame with all the relevant info from the buildings being plotted
tN = pd.concat([building_from_ind(buildings, i)[0] for i in range(len(buildings))])

b1 = building(building_from_ind(buildings, 0)[1])
t1 = building_from_ind(buildings, 0)[0]

In [ ]:
# Costruct the array of rays
O, D = construct_rays(t1,
                      sunpath,
                      offset_lower=[20, 8],
                      offset_upper=[8, 35],
                      di=1,
                      dj=1,
                      z=0)

In [ ]:
start = time.time()
calculate_rays(b1, O, D)
end = time.time()
print(f"Time taken for method 1: {end-start}")

In [ ]:
# Visualise the scene in 3D

plot_rays(b1, O, D, plot_ray=False, plot_shadow=True,s=0.5, figsize=(5,5), extent_offset=25)


In [ ]:

b.collision_coords = []
b.shadow_coords = []
b.origin_coords = []
b.sunny_coords = []  
b.shadow_con_hull = []

for i, O in enumerate(b.unique_vertices):
    ground_level = 0
    L = (ground_level-O[2])/sunpath[2]  # How many sunpaths do we need to move before we hit the ground?
    D = O + L*sunpath
    b.collision_coords.append(O)
    b.shadow_coords.append(D)
    b.origin_coords.append(O)
    b.sunny_coords.append([])
    
shadow = np.array(b.shadow_coords.copy())

redundant_dim = np.where(np.all(shadow == shadow[0,:], axis=0)==True)[0]
if len(redundant_dim) > 0:
    # Then one component can be totally removed from this vert - otherwise will break convex hull 
    shadow = np.delete(shadow, redundant_dim, axis=1)
else:
    print(f"Shape is not flat! Forcing anyway")
    redundant_dim = 2
    shadow = np.delete(shadow, redundant_dim, axis=1)
    

hull = ConvexHull(shadow)
hull_points = hull.simplices 

hull_points = shadow[hull.vertices]

# Find which specific coordinate was found to be redundant and removed from all coordinates
#removed_coord = np.full(len(hull_points),np.array(b.shadow_coords)[0][redundant_dim][0])

removed_coord = np.full(len(hull_points),0)

# Add it back in to the hull points
hull_points = np.column_stack((hull_points, removed_coord))
hull_points = [[tuple(point) for point in hull_points]]

b.shadow_con_hull = hull_points
b.shadow_coords = np.array(b.shadow_coords)

plot_rays_vertices(b, s=50, figsize=(5,5), extent_offset=20)

In [ ]:
%matplotlib notebook

b = building(building_from_ind(buildings, 0)[1])

b.unique_vertices = pd.DataFrame(np.concatenate(b.faces)).drop_duplicates().reset_index(drop=True).to_numpy()

N = 10

for face in b.faces:
    for i in range(0, len(face)-1):
        start = face[i]
        end = face[i+1]
        midpoints = np.linspace(start, end, N)
        b.unique_vertices = np.concatenate((b.unique_vertices, midpoints))
        #print(f"{b.unique_vertices}\n\n")

In [ ]:
from scipy.spatial import Delaunay
def alpha_shape(points, alpha, only_outer=True):
    """
    Compute the alpha shape (concave hull) of a set of points.
    :param points: np.array of shape (n,2) points.
    :param alpha: alpha value.
    :param only_outer: boolean value to specify if we keep only the outer border
    or also inner edges.
    :return: set of (i,j) pairs representing edges of the alpha-shape. (i,j) are
    the indices in the points array.
    """
    assert points.shape[0] > 3, "Need at least four points"
    def add_edge(edges, i, j):
        """
        Add an edge between the i-th and j-th points,
        if not in the list already
        """
        if (i, j) in edges or (j, i) in edges:
            # already added
            assert (j, i) in edges, "Can't go twice over same directed edge right?"
            if only_outer:
                # if both neighboring triangles are in shape, it's not a boundary edge
                edges.remove((j, i))
            return
        edges.add((i, j))
    tri = Delaunay(points)
    edges = set()
    # Loop over triangles:
    # ia, ib, ic = indices of corner points of the triangle
    for ia, ib, ic in tri.vertices:
        pa = points[ia]
        pb = points[ib]
        pc = points[ic]
        # Computing radius of triangle circumcircle
        # www.mathalino.com/reviewer/derivation-of-formulas/derivation-of-formula-for-radius-of-circumcircle
        a = np.sqrt((pa[0] - pb[0]) ** 2 + (pa[1] - pb[1]) ** 2)
        b = np.sqrt((pb[0] - pc[0]) ** 2 + (pb[1] - pc[1]) ** 2)
        c = np.sqrt((pc[0] - pa[0]) ** 2 + (pc[1] - pa[1]) ** 2)
        s = (a + b + c) / 2.0
        area = np.sqrt(s * (s - a) * (s - b) * (s - c))
        circum_r = a * b * c / (4.0 * area)
        if circum_r < alpha:
            add_edge(edges, ia, ib)
            add_edge(edges, ib, ic)
            add_edge(edges, ic, ia)
    return edges

In [ ]:
edges = alpha_shape(shadow, alpha=2)

fig, ax = plt.subplots(figsize=(18,4))
plt.subplot(1, 3, 1)
plt.plot(shadow[:, 0], shadow[:, 1], '.')
for i, j in edges:
    plt.plot(shadow[[i, j], 0], shadow[[i, j], 1])

plt.text(270.5,459, "alpha=1", size=18)


In [ ]:
from descartes import PolygonPatch
import alphashape

points = [(17, 158),(15, 135),(38, 183),(43, 19),(93, 88),(96, 140),(149, 163),(128, 248),(216, 265),(248, 210),(223, 167),(256, 151),(331, 214),(340, 187),(316, 53),(298, 35),(182, 0),(121, 42)]
points = [tuple(s) for s in shadow]

alpha = 0.2#0.95 * alphashape.optimizealpha(points)
hull = alphashape.alphashape(points, alpha)
hull_pts = hull.exterior.coords.xy
    
fig, ax = plt.subplots()
ax.scatter(hull_pts[0], hull_pts[1], color='red')
ax.add_patch(PolygonPatch(hull, fill=False, color='green'))
ax.plot(shadow[:,0], shadow[:,1], 'o')